In [10]:
import nltk
nltk.download('stopwords')

/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. D

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
import sys
import numpy as np, pandas as pd, re
from pprint import pprint

#Gensim
import gensim, spacy, logging, warnings
from gensim.utils import lemmatize, simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

#NLTK and stopwords
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import string
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from sklearn.base import TransformerMixin, BaseEstimator
import multiprocessing as mp
import re

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa


In [12]:
data_df =pd.read_csv('/content/drive/MyDrive/Review CLass/Reviews.csv')

In [13]:
data_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [14]:
data_df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [15]:
data_df = data_df.dropna()

In [16]:
df_bad_reviews = data_df[data_df['Score'] < 3]['Text']

In [17]:
#focussing on first thousand bad reviews only
df_bad_reviews = df_bad_reviews[0:1000]
df_bad_reviews = df_bad_reviews.reset_index(drop=True)

In [18]:
df_bad_reviews.head()

0    Product arrived labeled as Jumbo Salted Peanut...
1    If you are looking for the secret ingredient i...
2    My cats have been happily eating Felidae Plati...
3    I love eating them and they are good for watch...
4    The candy is just red , No flavor . Just  plan...
Name: Text, dtype: object

In [19]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
nlp = spacy.load('en_core_web_sm', disacle = ['ner', 'parser'])

In [21]:
def sent_to_words(sentences):
  for sent in sentences:
    sent = re.sub('\S*@\S*\s?', '', sent) #remove emails
    sent = re.sub(r'\b\w{1,3}\b', '', sent) #remove words less than three chars
    sent = re.sub('\s+', ' ', sent)  # remove newline chars
    sent = re.sub("\'", "", sent)  # remove single quotes
    sent = gensim.utils.simple_preprocess(str(sent), deacc=True)
    yield sent

In [22]:
#convert to list
data = df_bad_reviews.to_list()
data_words = list(sent_to_words(data))
print(data_words[1])

['looking', 'secret', 'ingredient', 'robitussin', 'believe', 'have', 'found', 'this', 'addition', 'root', 'beer', 'extract', 'ordered', 'which', 'good', 'made', 'some', 'cherry', 'soda', 'flavor', 'very', 'medicinal']


In [23]:
print(data_words[0])

['product', 'arrived', 'labeled', 'jumbo', 'salted', 'peanuts', 'peanuts', 'were', 'actually', 'small', 'sized', 'unsalted', 'sure', 'this', 'error', 'vendor', 'intended', 'represent', 'product', 'jumbo']


In [24]:
#buils bigrams and trigrams model
bigrams = gensim.models.Phrases(data_words, min_count=1, threshold=2) # higher threshold fewer phrases.
trigrams = gensim.models.Phrases(bigrams[data_words], threshold=2)

bigram_mod = gensim.models.phrases.Phraser(bigrams)
trigram_mod = gensim.models.phrases.Phraser(trigrams)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [25]:
def process_words(texts, stop_words = stop_words, allowed_postags= ['NOUN', "ADJ"]):
  """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
  texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
  texts = [bigram_mod[doc] for doc in texts]
  texts = [trigram_mod[bigram_mod[doc]]for doc in texts]
  text_out = []
  nlp = spacy.load('en_core_web_sm', disacle = ['ner', 'parser'])
  for sent in texts:
    doc = nlp(" ".join(sent))
    text_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  # remove stopwords once more after lemmatization
  text_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text_out]
  return text_out

In [26]:
data_ready = process_words(data_words)

In [27]:
data_ready[0]

['product',
 'jumbo',
 'peanut',
 'peanut',
 'small',
 'sized',
 'unsalted',
 'sure',
 'error',
 'vendor',
 'represent',
 'product',
 'jumbo']

In [28]:
#create dictionary
id2word = corpora.Dictionary(data_ready)

#create corpus:document term frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

#build base LDA Model
lda_model = gensim.models.LdaModel(corpus = corpus, num_topics=10, id2word=id2word, random_state=100, chunksize=100, passes=10, per_word_topics=True)
pprint(lda_model.print_topics())

[(0,
  '0.051*"coffee" + 0.018*"cocoa" + 0.018*"sure" + 0.017*"regular" + '
  '0.016*"stick" + 0.014*"candy" + 0.014*"waste" + 0.013*"weak" + 0.013*"diet" '
  '+ 0.012*"child"'),
 (1,
  '0.027*"review" + 0.025*"taste" + 0.022*"love" + 0.022*"drink" + '
  '0.020*"salt" + 0.015*"chip" + 0.015*"fact" + 0.014*"little" + '
  '0.014*"bottle" + 0.013*"brand"'),
 (2,
  '0.019*"packaging" + 0.015*"awful" + 0.015*"green" + 0.015*"flavor" + '
  '0.015*"wrong" + 0.014*"side" + 0.013*"bad" + 0.012*"water" + 0.012*"good" + '
  '0.011*"food"'),
 (3,
  '0.053*"food" + 0.033*"product" + 0.026*"ingredient" + 0.022*"flavor" + '
  '0.020*"pack" + 0.019*"great" + 0.017*"good" + 0.015*"year" + '
  '0.013*"company" + 0.012*"real"'),
 (4,
  '0.075*"product" + 0.029*"cookie" + 0.021*"bean" + 0.017*"problem" + '
  '0.017*"plastic" + 0.016*"free" + 0.013*"chicken" + 0.013*"one" + '
  '0.012*"quality" + 0.012*"meat"'),
 (5,
  '0.030*"food" + 0.024*"chocolate" + 0.023*"taste" + 0.021*"water" + '
  '0.018*"product"

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.110948075569757


In [30]:
cohrence_model_lda = CoherenceModel(model=lda_model, texts = data_ready, dictionary=id2word, coherence='c_v')

In [31]:
cohrence_model_lda.get_coherence()

0.354299916175867

Now that we have the baseline coherence score for the default LDA model, let’s perform a series of sensitivity tests to help determine the following model hyperparameters:
Number of Topics (K)
Dirichlet hyperparameter alpha: Document-Topic Density
Dirichlet hyperparameter beta: Word-Topic Density

In [34]:
#lets do some hyperparameter tuning to get good coherence
def compute_coherence_values(corpus, data_ready, dictionary, k, a, b):

  lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word= dictionary, num_topics=k, random_state=100, chunksize=100, passes=10, alpha=a, eta=b)
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_ready, dictionary=id2word, coherence='c_v')
  return coherence_model_lda.get_coherence()

In [37]:
import numpy as np
import tqdm

grid = {}
grid['Validation_set'] = {}

#topic range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

#alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

#beta parameter
beta = list(np.arange(0.01,1, 0.3))
beta.append('symmetric')

#validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],'Topics': [],'Alpha': [], 'Beta': [],'Coherence': []}
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus,data_ready=data_ready ,dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()




  0%|          | 0/540 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)



  0%|          | 1/540 [00:06<1:02:48,  6.99s/it]


  0%|          | 2/540 [00:14<1:03:43,  7.11s/it]


  1%|          | 3/540 [00:22<1:07:04,  7.49s/it]


  1%|          | 4/540 [00:31<1:09:18,  7.76s/it]


  1%|          | 5/540 [00:38<1:08:04,  7.63s/it]


  1%|          | 6/540 [00:45<1:05:55,  7.41s/it]


  1%|▏         | 7/540 [00:53<1:07:38,  7.61s/it]


  1%|▏         | 8/540 [01:03<1:12:43,  8.20s/it]


  2%|▏         | 9/540 [01:13<1:17:54,  8.80s/it]


  2%|▏         | 10/540 [01:22<1:17:43,  8.80s/it]


  2%|▏         | 11/540 [01:28<1:11:56,  8.16s/it]


  2%|▏         | 12/540 [01:36<1:11:55,  8.17s/it]


  2%|▏         | 13/540 [01:46<1:15:07,  8.55s/it]


  3%|▎         | 14/540 [01:56<1:18:33,  8.96s/it]


  3%|▎         | 15/540 [02:05<1:18:37,  8.99s/it]


  3%|▎        

In [40]:
lda_df = pd.read_csv('/content/lda_tuning_results.csv')
lda_df.head(5)

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.234464
1,75% Corpus,2,0.01,0.31,0.240983
2,75% Corpus,2,0.01,0.61,0.278622
3,75% Corpus,2,0.01,0.9099999999999999,0.253921
4,75% Corpus,2,0.01,symmetric,0.268649


In [45]:
lda_df = lda_df.sort_values(by = 'Coherence', ascending=False)

In [46]:
lda_df.head()

,Validation_Set,Topics,Alpha,Beta,Coherence
258,75% Corpus,10,0.9099999999999999,0.9099999999999999,0.518514
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.488778
527,100% Corpus,10,0.9099999999999999,0.61,0.453042
257,75% Corpus,10,0.9099999999999999,0.61,0.452593
177,75% Corpus,7,asymmetric,0.61,0.444297


In [32]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.9,
                                           eta=0.6)

In [1]:
!pip install pyLDAvis

In [2]:
!pip install matplotlib

In [33]:
# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
import pyLDAvis.gensim_models as gensimvis

In [35]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.173478 -0.011483       1        1  32.784459
8      0.000722  0.102001       2        1  12.367516
4      0.004491 -0.015095       3        1  12.308041
5     -0.028892 -0.011865       4        1   7.241278
1     -0.026716 -0.019888       5        1   7.190425
6     -0.022143 -0.011407       6        1   7.099829
0     -0.029885 -0.013787       7        1   6.424215
2     -0.024900 -0.011962       8        1   5.364884
7     -0.024302 -0.002931       9        1   4.822911
9     -0.021853 -0.003584      10        1   4.396439, topic_info=             Term        Freq       Total Category  logprob  loglift
3         product  218.000000  218.000000  Default  30.0000  30.0000
25           food  124.000000  124.000000  Default  29.0000  29.0000
211          time   47.000000   47.000000  Default  28.0000  28.0000
154        cookie   28.000000   28.000000  Default  27.0000  27.0000
408          item   38.000000   38.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
2360  wanted_food    0.518540    2.615647  Topic10  -7.1910   1.5061
2361   weeks_back    0.518540    2.615647  Topic10  -7.1910   1.5061
2358  unfortunate    0.681861    5.090201  Topic10  -6.9172   1.1141
1212         back    0.688940    5.626867  Topic10  -6.9068   1.0242
456       garbage    0.604523    8.818913  Topic10  -7.0375   0.4442

[434 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2118      9  0.325745    acceptable
832       1  0.437671    aftertaste
832       6  0.087534    aftertaste
832       8  0.262603    aftertaste
3737      8  0.334609  amalgamation
...     ...       ...           ...
279       7  0.160459         wrong
31        1  0.831257          year
31        2  0.122244          year
31        3  0.024449          year
31        7  0.024449          year

[738 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 5, 6, 2, 7, 1, 3, 8, 10])

In [36]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [37]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.4480,"salt, chip, store, item, cake, bag, stick, siz...","[product, jumbo, peanut, peanut, small, sized,..."
1,1,3.0,0.3302,"product, food, good, taste, flavor, ingredient...","[secret, ingredient, robitussin, addition, bee..."
2,2,3.0,0.5149,"product, food, good, taste, flavor, ingredient...","[cat, year, shape, food, different, food, firs..."
3,3,3.0,0.2974,"product, food, good, taste, flavor, ingredient...","[good, movie, sweet, transfer, lock, baggie, f..."
4,4,4.0,0.1513,"salt, chip, store, item, cake, bag, stick, siz...","[candy, flavor, plan, chewy]"
5,5,3.0,0.2429,"product, food, good, taste, flavor, ingredient...","[good, soft]"
6,6,4.0,0.1949,"salt, chip, store, item, cake, bag, stick, siz...","[day, bag]"
7,7,3.0,0.2765,"product, food, good, taste, flavor, ingredient...","[mango, various, sport, nutrition, product, de..."
8,8,8.0,0.4838,"product, time, cookie, order, item, return, ch...","[sweetener, sugar, undigestible, body, many, c..."
9,9,0.0,0.1000,"organic, pure, enough, lemon, skin, life, meal...",[]
